In [3]:
import clickhouse_connect


# 🔗 Conexão com ClickHouse
client = clickhouse_connect.get_client(
    host="localhost",
    port=8123,
    username="default",
    password="afonsystem"
)


def table_exists(table_name):
    result = client.query(f"EXISTS TABLE {table_name}")
    return result.result_rows[0][0] == 1


# 🧹 Função para remover duplicatas da tabela commits
def remove_duplicate_commits():
    if not table_exists("commits"):
        print("❌ A tabela 'commits' não existe.")
        return

    print("🚀 Limpando duplicatas na tabela 'commits'...")

    client.command("""
        CREATE TABLE commits_deduplicated AS
        SELECT *
        FROM (
            SELECT *,
                   row_number() OVER (PARTITION BY repo_name, sha ORDER BY date DESC) AS rn
            FROM commits
        )
        WHERE rn = 1
    """)

    client.command("RENAME TABLE commits TO commits_old, commits_deduplicated TO commits")
    client.command("DROP TABLE commits_old")

    client.command("OPTIMIZE TABLE commits FINAL")

    print("✅ Duplicatas na tabela 'commits' removidas com sucesso.\n")


# 🧹 Função para remover duplicatas da tabela pull_requests
def remove_duplicate_pull_requests():
    if not table_exists("pull_requests"):
        print("❌ A tabela 'pull_requests' não existe.")
        return

    print("🚀 Limpando duplicatas na tabela 'pull_requests'...")

    client.command("""
        CREATE TABLE pull_requests_deduplicated AS
        SELECT *
        FROM (
            SELECT *,
                   row_number() OVER (PARTITION BY repo_name, number ORDER BY created_at DESC) AS rn
            FROM pull_requests
        )
        WHERE rn = 1
    """)

    client.command("RENAME TABLE pull_requests TO pull_requests_old, pull_requests_deduplicated TO pull_requests")
    client.command("DROP TABLE pull_requests_old")

    client.command("OPTIMIZE TABLE pull_requests FINAL")

    print("✅ Duplicatas na tabela 'pull_requests' removidas com sucesso.\n")


# 🚀 Função principal
if __name__ == "__main__":
    print("🧠 Iniciando processo de deduplicação no ClickHouse...")

    try:
        remove_duplicate_commits()
    except Exception as e:
        print(f"❌ Erro ao remover duplicatas da tabela commits: {e}")

    try:
        remove_duplicate_pull_requests()
    except Exception as e:
        print(f"❌ Erro ao remover duplicatas da tabela pull_requests: {e}")

    print("🏁 Processo de deduplicação concluído com sucesso!")


🧠 Iniciando processo de deduplicação no ClickHouse...
🚀 Limpando duplicatas na tabela 'commits'...
❌ Erro ao remover duplicatas da tabela commits: HTTPDriver for http://localhost:8123 received ClickHouse error code 42
 Code: 42. DB::Exception: ORDER BY or PRIMARY KEY clause is missing. Consider using extended storage definition syntax with ORDER BY or PRIMARY KEY clause. With deprecated old syntax (highly not recommended) storage MergeTree requires 3 to 4 parameters: 
name of column with date,
[sampling element of primary key],
primary key expression,
index granularity

Syntax for the MergeTree table engine:

CREATE TABLE [IF NOT EXISTS] [db.]table_name [ON CLUSTER cluster]
(
    name1 [type1] [DEFAULT|MATERIALIZED|ALIAS expr1] [TTL expr1],
    name2 [type2] [DEFAULT|MATERIALIZED|ALIAS expr2] [TTL expr2],
    ...
    INDEX index_name1 expr1 TYPE type1(...) [GRANULARITY value1],
    INDEX index_name2 expr2 TYPE type2(...) [GRANULARITY value2]
) ENGINE = MergeTree()
ORDER BY expr
[PARTIT